In [1]:
import numpy as np
import pandas as pd

In [88]:
import mpmath

# Параметры 

In [2]:
L = 1.5 # длина в м
R = 1.5 # радиус
E = 7e10  # модуль Юнга
ro = 2700  # плотоность алюминия
ro_air = 1.2  # в салоне
nu = 0.33  # коэффицент Пуассона
G = E / (2 * (1 + nu))  # модуль сдвига
h = 0.0015   # тодщина пластины
D = (E * h ** 3) / (12 * (1 - nu ** 2))  # жесткость из уравнения колебания
beta = 0.01 # - коэф потерь
c = 330 # в салоне

# Данные Bies

In [3]:
df = pd.read_csv('Bies_TBL_Data.csv')
df['P^2'] = 4 * 10 ** (df['Lterz'] / 10 - 10)
df['w'] = 2 * np.pi * df['Freq']

In [4]:
def get_params(ind):
    return dict(df.iloc[ind])

# Функции

Векторизуем по m1 и m2 - они могут быть векторами. 

In [5]:
def int1(L, m1, lam1, w, Uph):
    num1 = (((-1) ** (m1 + 1)) * ((np.pi * L * m1) ** 2) * np.exp(-L/lam1 - 1j*L*w/Uph) + (L ** 5) * (Uph + 1j*lam1*w)**3 / 
            (2 * (Uph * lam1) ** 3) + (np.pi ** 2) * (L*m1)**3 / (2*lam1) + (1j * ((np.pi * m1) ** 2) * w * L **3) / (2 * Uph) +
           (np.pi * L * m1) ** 2)
    den1 = ((L**2) * (1/lam1 + 1j*w/Uph)**2 + (np.pi * m1) ** 2) ** 2
    
    num2 = (((-1) ** (m1 + 1)) * ((np.pi * L * m1) ** 2) * np.exp(-L/lam1 + 1j*L*w/Uph) + (L ** 5) * (Uph - 1j*lam1*w)**3 / 
            (2 * (Uph * lam1) ** 3) + (np.pi ** 2) * (L*m1)**3 / (2*lam1) - (1j * ((np.pi * m1) ** 2) * w * L **3) / (2 * Uph) +
           (np.pi * L * m1) ** 2)
    den2 = ((L**2) * (1/lam1 - 1j*w/Uph)**2 + (np.pi * m1) ** 2) ** 2
    return num1 / den1 + num2 / den2

In [6]:
def int2(R, m2, lam2):
    return (2 * np.pi * lam2 * R ** 5 - 2 * (R ** 4) * lam2 ** 2 + 2 * (R ** 4) * (lam2 ** 2) * np.exp(-2*np.pi * R / lam2) + 
           2*np.pi*((R*lam2) ** 3) * (m2)**2) / (R**2 + (lam2*m2)**2) **2

In [7]:
def octave_width(w):
    return 0.23 * w

In [8]:
# w - циклическая
def full_int(m1, m2, lam1, lam2, w, Uph, p_sq):
    coef = (4 / (np.pi * R * L) ** 2) * (p_sq / octave_width(w))
    return coef * np.real(np.outer(int1(L, m1, lam1, w, Uph), int2(R, m2, lam2)))

In [9]:
# m1 - вниз, m2 - вправо
def B_coef(m1, m2, lam1, lam2, w, Uph, p_sq):
    m1v, m2v = np.meshgrid(m1, m2, indexing='ij')
    k1 = m1v * np.pi / L
    k2 = m2v / R
    den = (D * (k1 ** 2 + k2 ** 2) ** 2 + (E * h * k1 ** 4) / ((R ** 2) * (k1 ** 2 + k2 ** 2) ** 2) - ro * h * (w ** 2)) ** 2 + (beta * w) ** 2
    return full_int(m1, m2, lam1, lam2, w, Uph, p_sq) / den

# Считаем поточечно 

In [ ]:
def func_sphere(k1, k1, lam1, lam2, w, Uph, p_sq):
    # rewrite for sphere

In [18]:
def func(phi, r, lam1, lam2, w, Uph, p_sq):
    m1 = np.arange(1, 23)
    m2 = np.arange(1, 23)
    k = w / c
    coef = (w ** 2) * 8 * ((np.pi * L * m1) ** 2) * (R ** 4) * (r ** 3)
    num1 = ((-1) ** (m1 + 1)) * np.cos(L * r * np.cos(phi)) + 1
    num2 = np.sin(phi) ** 2
    num3 = np.sin(np.pi * R * r * np.sin(phi)) ** 2
    den1 = np.sqrt(k ** 2 - r ** 2)
    den2 = ((L * r * np.cos(phi)) ** 2 - (np.pi * m1) ** 2) ** 2
    den3 = ((R * r * np.sin(phi)) ** 2 - m2 ** 2) ** 2
    num = coef * num1 * num2 * num3
    den = den1 * den2 * den3
    den = 1 / den
    b = B_coef(m1, m2, lam1, lam2, w, Uph, p_sq)
    return np.sum(b * (np.outer(coef * num, den)), keepdims=False)

In [19]:
def vec_func(phi, r, lam1, lam2, w, Uph, p_sq):
    return np.array([func(f, rad, lam1, lam2, w, Uph, p_sq) for f, rad in zip(phi, r)])

In [33]:
func(0, 0, lam1, lam2, w, Uph, p_sq)

0.0

In [21]:
import quadpy

In [93]:
__, lam1, lam2, __, p_sq, Uph, w = get_params(0).values()

In [82]:
scheme = quadpy.c2.get_good_scheme(21)
val = scheme.integrate(
    lambda x: vec_func(x[0], x[1], lam1, lam2, w, Uph, p_sq),
    quadpy.c2.rectangle_points([0, 2*np.pi], [0, w/c]),
)

In [83]:
val

1585633.0284960882

In [84]:
10 * np.log10(val * (ro_air * w) / 2)

102.74844356055128